40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

In [1]:
import re

In [2]:
# 区切り文字
separator = re.compile('\t|,')

# 係り受け
dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
                            (-?\d+)       # 数字(係り先)
                          ''', re.VERBOSE)

In [3]:
class Morph:
    def __init__(self, line):
        
        #タブとカンマで分割
        cols = separator.split(line)
        
        self.surface = cols[0] # 表層形(surface)
        self.base = cols[7]    # 基本形(base)
        self.pos = cols[1]     # 品詞(pos)
        self.pos1 = cols[2]    # 品詞細分類1(pos1)

In [4]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.srcs = []   # 係り元文節インデックス番号のリスト
        self.dst  = dst  # 係り先文節インデックス番号
        self.phrase = ''.join([morph.surface for morph in morphs]) # 文節

In [5]:
# 係り元を代入し、Chunkリストを文のリストを追加
def append_sentence(chunks, sentences):
    
    # 係り元を代入
    for i, chunk in enumerate(chunks):
        if chunk.dst != -1:
            chunks[chunk.dst].srcs.append(i)
    sentences.append(chunks)
    return sentences, []

カラム	意味
1	先頭カラムは*。係り受け解析結果であることを示す。
2	文節番号（0から始まる整数）
3	係り先番号＋D
4	主辞/機能語の位置と任意の個数の素性列
5	係り関係のスコア。係りやすさの度合で、一般に大きな値ほど係りやすい。

In [6]:
%time

morphs = []
chunks = []
sentences = []

with open('./neko.txt.cabocha') as f:
    
    for line in f:
        dependancies = dependancy.match(line)
        
        # EOSまたは係り受け解析結果でない場合
        if not (line == 'EOS\n' or dependancies):
            morphs.append(Morph(line))
            
        # EOSまたは係り受け解析結果で、形態素解析結果がある場合
        elif len(morphs) > 0:
            chunks.append(Chunk(morphs, dst))
            morphs = []
       
        # 係り受け結果の場合
        if dependancies:
            dst = int(dependancies.group(1))
        
        # EOSで係り受け結果がある場合
        if line == 'EOS\n' and len(chunks) > 0:
            sentences, chunks = append_sentence(chunks, sentences)

CPU times: user 106 µs, sys: 0 ns, total: 106 µs
Wall time: 117 µs


In [7]:
for i, chunk in enumerate(sentences[7]):
    print('{}: {}, 係り先:{}, 係り元:{}'.format(i, chunk.phrase, chunk.dst, chunk.srcs))

0: この, 係り先:1, 係り元:[]
1: 書生というのは, 係り先:7, 係り元:[0]
2: 時々, 係り先:4, 係り元:[]
3: 我々を, 係り先:4, 係り元:[]
4: 捕えて, 係り先:5, 係り元:[2, 3]
5: 煮て, 係り先:6, 係り元:[4]
6: 食うという, 係り先:7, 係り元:[5]
7: 話である。, 係り先:-1, 係り元:[1, 6]
